In [216]:
# 不要な警告を非表示にする
import warnings
warnings.filterwarnings('ignore')

## 著者分類器 (夏目漱石, 森鴎外, 芥川龍之介, 宮沢賢治)

青空文庫のデータを訓練データとして、モデルを生成し、ある文がどの著者によるものか、あるいは著者っぽさを判定する。

In [225]:
# 必要なライブラリのimport
# 最後にCNNを用いてモデルを生成するため、オープンソースニューラスネットワークライブラリであるKerasをimportしている
import sys, os.path, re, csv, os, glob
import pandas as pds
import numpy as np
import zipfile
import codecs
from keras.layers import Activation, Dense, Dropout, Flatten, Convolution2D, MaxPooling2D, Reshape, Input, concatenate
from keras.models import Model, Sequential
from keras.layers.embeddings import Embedding
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from keras.optimizers import Adam
from keras.callbacks import LearningRateScheduler, Callback, CSVLogger, ModelCheckpoint

# ワーキングディレクトリの設定
base_url = "http://www.aozora.gr.jp/"
data_dir = "./"
aozora_dir = data_dir + "aozora_data/"
log_dir = aozora_dir + "log/"
target_author_file = data_dir + "target_author.csv"

auth_target = []
with open(target_author_file,"r") as f:
    reader = csv.reader(f)
    for row in reader:
        auth_target.append(row)

# ターゲットとする著者と作品一覧が載っているURLのリスト
auth_target


[['author', ' "url"'],
 ['natsume',
  ' http://www.aozora.gr.jp/index_pages/person148.html#sakuhin_list_1'],
 ['akutagawa',
  ' http://www.aozora.gr.jp/index_pages/person879.html#sakuhin_list_1'],
 ['mori',
  ' http://www.aozora.gr.jp/index_pages/person129.html#sakuhin_list_1'],
 ['miyazawa',
  ' https://www.aozora.gr.jp/index_pages/person81.html#sakuhin_list_1']]

### 教師データの生成

In [226]:
# 各著者についてラベル付きの文データを生成する
def author_data_integ(auth_target=auth_target):
    for w in auth_target[1:]:
        print ("starting: " + w[0])
        auth_dir = '{}{}/'.format(aozora_dir, w[0])
        csv_dir = '{}{}'.format(auth_dir, "csv/")
        files = os.listdir(csv_dir)
        integ_np = np.array([["author", "line"]])
        for file in files:
            if "csv" in file:
                print ("   now at: " + file)
                file_name = csv_dir + file
                pds_data = pds.read_csv(file_name, index_col=0)
                pds_data = pds_data.dropna()
                np_data = np.array(pds_data.ix[:,[0,2]])

                out = [j for j in range(len(np_data)) if '-----------' in str(np_data[j,1])]
                if not out: out = [1]
                hyphen_pos = int(out[len(out) - 1])

                last_20 = len(np_data) - 20

                np_data = np_data[hyphen_pos+1:last_20,:]
                integ_np = np.vstack((integ_np, np_data))

        integ_pds = pds.DataFrame(integ_np[1:,:], columns=integ_np[0,:])
        integ_pds.to_csv(auth_dir + w[0] + '_integ.csv', quoting=csv.QUOTE_ALL)
        print ("finished: " + w[0])
        
def load_integ(author, auth_dir):
    integ_csv = auth_dir + author + "_integ.csv"
    data = pds.read_csv(integ_csv)
    return data

natsume_data = load_integ(auth_target[1][0], auth_dir = '{}{}/'.format(aozora_dir, auth_target[1][0]))
np_natsume = np.array(natsume_data.ix[1:,1:])

akutagawa_data = load_integ(auth_target[2][0], auth_dir = '{}{}/'.format(aozora_dir, auth_target[2][0]))
np_akutagawa = np.array(akutagawa_data.ix[1:,1:])

mori_data = load_integ(auth_target[3][0], auth_dir = '{}{}/'.format(aozora_dir, auth_target[3][0]))
np_mori = np.array(mori_data.ix[1:,1:])

miyazawa_data = load_integ(auth_target[4][0], auth_dir = '{}{}/'.format(aozora_dir, auth_target[4][0]))
np_miyazawa = np.array(miyazawa_data.ix[1:,1:])

natsume_txt = np.array([np_natsume[:,1]]).T
akutagawa_txt = np.array([np_akutagawa[:,1]]).T
mori_txt = np.array([np_mori[:,1]]).T
miyazawa_txt = np.array([np_miyazawa[:,1]]).T

natsume_id = np.array([np.zeros(len(np_natsume))]).T
akutagawa_id = np.array([np.zeros(len(np_akutagawa)) + 1]).T
mori_id = np.array([np.zeros(len(np_mori)) + 2]).T
miyazawa_id = np.array([np.zeros(len(np_miyazawa)) + 3]).T

natsume = np.hstack((natsume_txt, natsume_id))
akutagawa = np.hstack((akutagawa_txt, akutagawa_id))
mori = np.hstack((mori_txt, mori_id))
miyazawa = np.hstack((miyazawa_txt, miyazawa_id))

# サンプル数の比較
print (len(natsume), len(akutagawa), len(mori), len(miyazawa))

105372 63536 109323 50871


In [227]:
natsume

array([['亡友の記念だと思つて長い間それを袋の中に入れて仕舞つて置いた。', 0.0],
       ['年數の經つに伴れて、ある時は丸で袋の所在を忘れて打ち過ぎる事も多かつた。', 0.0],
       ['近頃｜不圖思ひ出して、あゝして置いては轉宅の際などに何處へ散逸するかも知れないから、今のうちに表具屋へ遣つて懸物にでも仕立てさせやうと云ふ氣が起つた。',
        0.0],
       ...,
       ['あれは胡瓜を擦ったんです。', 0.0],
       ['患者さんが足が熱って仕方がない、胡瓜の汁で冷してくれとおっしゃるもんですから私が始終擦って上げました」', 0.0],
       ['「じゃやっぱり大根おろしの音なんだね」', 0.0]], dtype=object)

In [228]:
akutagawa

array([['その次ぎには、その小説の中に描かれた生活に憧憬を持つてゐる。', 1.0],
       ['これには時々不思議な気持がしないことはない。', 1.0],
       ['現に僕の知つてゐる或る人などは随分経済的に苦しい暮らしをしてゐながら、富豪や華族ばかり出て来る通俗小説を愛読してゐる。',
        1.0],
       ...,
       ['君の弟が、ステッキをふりまわして「兄さん万歳」を連叫する。', 1.0],
       ['――それが、いよいよ、君が全く見えなくなるまで、続いた。', 1.0],
       ['帰りぎわに、ふりむいて見たら、例の年よりの異人は、まだ、ぼんやり船の出て行った方をながめている。', 1.0]],
      dtype=object)

In [229]:
mori

array([['どうかして間違って二度話し掛けて、その客に「ひゅうひゅうと云うのだろう」なんぞと、先を越して云われようものなら、お金の悔やしがりようは一通りではない。',
        2.0],
       ['なぜと云うに、あの女は一度来た客を忘れると云うことはないと云って、ひどく自分の記憶を恃んでいたからである。', 2.0],
       ['それを客の方から頼んで二度話して貰ったものは、恐らくは僕一人であろう。', 2.0],
       ...,
       ['住いは六十五のとき下谷徒士町に移り、六十七のとき一時藩の上邸に入っていて、麹町一丁目半蔵門外の壕端の家を買って移った。',
        2.0],
       ['策士｜雲井龍雄と月見をした海嶽楼は、この家の二階である。', 2.0],
       ['幕府滅亡の余波で、江戸の騒がしかった年に、仲平は七十で表向き隠居した。', 2.0]], dtype=object)

In [230]:
miyazawa

array([['荒き渚のあけがたを', 3.0],
       ['家長は白きもんぱして', 3.0],
       ['こらをはげまし急ぎくる', 3.0],
       ...,
       ['※〔〕付きの表題は、底本編集時におぎなわれたものです。', 3.0],
       ['入力：junk', 3.0],
       ['校正：土屋隆', 3.0]], dtype=object)

### Ngramによる言語モデルの生成

In [231]:
data_integ = np.vstack((np.vstack((np.vstack((natsume, akutagawa)),mori)),miyazawa))

In [232]:
import ngram

# ngramと全文検索を用いた分類器
def ngram_search_c12n(raw_txt):
    all_data = [natsume, mori, akutagawa, miyazawa]
    target_author = ["夏目漱石","森鴎外","芥川龍之介","宮沢賢治"]
    score_list = []
    for data in all_data:
        G = ngram.NGram(data[:,0].tolist())
        score_list.append(G.search(raw_txt)[0][1])
    return target_author[score_list.index(max(score_list))]

In [233]:
raw_txt = """
もうけつしてさびしくはない
なんべんさびしくないと云つたとこで
またさびしくなるのはきまつてゐる
"""
ngram_search_c12n(raw_txt)

'宮沢賢治'

In [234]:
raw_txt = """
ぼんやりとした不安
"""
ngram_search_c12n(raw_txt)

'宮沢賢治'

In [235]:
raw_txt = """
のどかな春の日を

鳴き尽くし、鳴きあかし、

また鳴き暮らさなければ気が済まんと見える。

その上どこまでも登って行く、

いつまでも登って行く。

雲雀はきっと雲の中で死ぬに相違ない。

登り詰めた揚句は、

流れて雲に入って、

漂うているうちに形は消えてなくなって、

ただ声だけが空の裡に残るのかもしれない。
"""
ngram_search_c12n(raw_txt)

'夏目漱石'

In [236]:
raw_txt = """
ロシアのビザめんどくさいねえ
"""
ngram_search_c12n(raw_txt)

'森鴎外'

In [174]:
data_integ[:,0]

array(['亡友の記念だと思つて長い間それを袋の中に入れて仕舞つて置いた。',
       '年數の經つに伴れて、ある時は丸で袋の所在を忘れて打ち過ぎる事も多かつた。',
       '近頃｜不圖思ひ出して、あゝして置いては轉宅の際などに何處へ散逸するかも知れないから、今のうちに表具屋へ遣つて懸物にでも仕立てさせやうと云ふ氣が起つた。',
       ..., '※〔〕付きの表題は、底本編集時におぎなわれたものです。', '入力：junk', '校正：土屋隆'],
      dtype=object)

In [175]:
natsume[:,0]

array(['亡友の記念だと思つて長い間それを袋の中に入れて仕舞つて置いた。',
       '年數の經つに伴れて、ある時は丸で袋の所在を忘れて打ち過ぎる事も多かつた。',
       '近頃｜不圖思ひ出して、あゝして置いては轉宅の際などに何處へ散逸するかも知れないから、今のうちに表具屋へ遣つて懸物にでも仕立てさせやうと云ふ氣が起つた。',
       ..., 'あれは胡瓜を擦ったんです。',
       '患者さんが足が熱って仕方がない、胡瓜の汁で冷してくれとおっしゃるもんですから私が始終擦って上げました」',
       '「じゃやっぱり大根おろしの音なんだね」'], dtype=object)

In [179]:
import MeCab
mecab = MeCab.Tagger("-Owakati")

def mecab_parse(s):
    return mecab.parse(str(s)).strip().split(" ")

### CNNによる学習

In [71]:
data_integ = np.vstack((np.vstack((np.vstack((natsume, akutagawa)),mori)),miyazawa))

In [72]:
def load_data(txt, max_length=200):
    txt_list = []
    for l in txt:
        txt_line = [ord(x) for x in str(l).strip()]
        # You will get encoded text in array, just like this
        # [25991, 31456, 12391, 12399, 12394, 12367, 12387, 12390, 23383, 24341, 12391, 12354, 12427, 12290]
        txt_line = txt_line[:max_length]
        txt_len = len(txt_line)
        if txt_len < max_length:
            txt_line += ([0] * (max_length - txt_len))
        txt_list.append((txt_line))
    return txt_list

In [76]:
txt_list = load_data(txt = data_integ[:,0])
np_txt = np.array(txt_list)

tgt_list = data_integ[:,1]
np_tgt_list = np_utils.to_categorical(tgt_list)

In [77]:
np_txt

array([[20129, 21451, 12398, ...,     0,     0,     0],
       [24180, 25976, 12398, ...,     0,     0,     0],
       [36817, 38915, 65372, ...,     0,     0,     0],
       ...,
       [ 8251, 12308, 12309, ...,     0,     0,     0],
       [20837, 21147, 65306, ...,     0,     0,     0],
       [26657, 27491, 65306, ...,     0,     0,     0]])

In [79]:
tgt_list

array([0.0, 0.0, 0.0, ..., 3.0, 3.0, 3.0], dtype=object)

In [89]:
len(np_tgt_list)

329102

In [90]:
len(np_txt)

329102

In [142]:
txt_list = load_data(txt = data_integ[:,0])
np_txt = np.array(txt_list)

tgt_list = data_integ[:,1]
np_tgt_list = np_utils.to_categorical(tgt_list)

In [99]:
def create_model(embed_size=128, max_length=200, filter_sizes=(2, 3, 4, 5), filter_num=64):
    inp = Input(shape=(max_length,))
    emb = Embedding(0xffff, embed_size)(inp)
    emb_ex = Reshape((max_length, embed_size, 1))(emb)
    
    convs = []
    for filter_size in filter_sizes:
        # 畳み込み層
        conv = Convolution2D(filter_num, filter_size, embed_size, activation="relu")(emb_ex)
        # プーリング層
        pool = MaxPooling2D(pool_size=(max_length - filter_size + 1, 1))(conv)
        convs.append(pool)
    # concatenateで層をマージ
    convs_merged = concatenate(convs)
    reshape = Reshape((filter_num * len(filter_sizes),))(convs_merged)
    fc1 = Dense(64, activation="relu")(reshape) # 全結合ニューラルネットワークレイヤー(64次元)、ReLU関数
    bn1 = BatchNormalization()(fc1) # 隠れ層に対する標準化
    do1 = Dropout(0.5)(bn1) # ドロップアウト（過学習を避ける）
    fc2 = Dense(4, activation='sigmoid')(do1) # 全結合ニューラルネットワークレイヤー(4次元)、シグモイド関数
    model = Model(input=inp, output=fc2)# モデルの生成
    return model

def train(inputs, targets, batch_size=100, epoch_count=4, max_length=200, model_filepath=aozora_dir + "model.h5", learning_rate=0.001):
    start = learning_rate #学習率
    stop = learning_rate * 0.01
    learning_rates = np.linspace(start, stop, epoch_count) # 学習率の等差数列

    model = create_model(max_length=max_length) # モデルを生成
    optimizer = Adam(lr=learning_rate) # アダムオプティマイザによる最適化
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])  # モデルのコンパイル
    model.summary()
    
    # モデルをファイルとして保存しておく
    target = os.path.join('/tmp', 'weights.*.hdf5')
    files = [(f, os.path.getmtime(f)) for f in glob.glob(target)]
    if len(files) != 0:
        latest_saved_model = sorted(files, key=lambda files: files[1])[-1]
        model.load_weights(latest_saved_model[0])
    csv_logger_file = '/tmp/clcnn_training.log'
    checkpoint_filepath = "/tmp/weights.{epoch:02d}-{loss:.2f}-{acc:.2f}-{val_loss:.2f}-{val_acc:.2f}.hdf5"

    model.fit(inputs, targets,
              nb_epoch=epoch_count,
              batch_size=batch_size,
              verbose=1,
              validation_split=0.1,
              shuffle=True,
              callbacks=[
                  LearningRateScheduler(lambda epoch: learning_rates[epoch]),
                  CSVLogger(csv_logger_file),
                  ModelCheckpoint(filepath=checkpoint_filepath, verbose=1, save_best_only=True, save_weights_only=False, monitor='val_acc')
              ]) # モデルを学習させる、inputs、targetsがそれぞれデータとラベル

    model.save(model_filepath) # モデルをファイルに保存




if __name__ == "__main__":
    train(np_txt, np_tgt_list)


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_15 (InputLayer)           (None, 200)          0                                            
__________________________________________________________________________________________________
embedding_15 (Embedding)        (None, 200, 128)     8388480     input_15[0][0]                   
__________________________________________________________________________________________________
reshape_17 (Reshape)            (None, 200, 128, 1)  0           embedding_15[0][0]               
__________________________________________________________________________________________________
conv2d_56 (Conv2D)              (None, 199, 1, 64)   16448       reshape_17[0][0]                 
__________________________________________________________________________________________________
conv2d_57 

In [125]:
import numpy as np
import pandas as pds
from keras.models import load_model

model_file = "/tmp/weights.01-0.26-0.90-1.61-0.56.hdf5"
target_author = ["夏目漱石","芥川龍之介","森鴎外","宮沢賢治"]

In [126]:
# Encodes the raw_txt
def text_encoding(raw_txt):
    txt = [ord(x) for x in str(raw_txt).strip()]
    txt = txt[:200]
    if len(txt) < 200:
        txt += ([0] * (200 - len(txt)))
    return txt

In [127]:
# Predict
def predict(comments, model_filepath="tmp/model.h5"):
    model = load_model(model_filepath)
    ret = model.predict(comments)
    return ret

In [128]:
def result(raw_txt):
    txt = text_encoding(raw_txt)
    predict_result = predict(np.array([txt]), model_filepath=model_file)
    pds_predict_result = pds.DataFrame(predict_result, columns=target_author)
    print(pds_predict_result)
    return target_author[np.argmax(pds_predict_result.as_matrix())]

In [129]:
raw_txt = "隴西の李徴は博學才穎、天寶の末年、若くして名を虎榜に連ね、ついで江南尉に補せられたが、性、狷介、自ら恃む所頗る厚く、賤吏に甘んずるを潔しとしなかつた。"
result(raw_txt)

      夏目漱石     芥川龍之介       森鴎外          宮沢賢治
0  0.00001  0.000025  0.001205  4.801459e-08


'森鴎外'

In [131]:
raw_txt = """
のどかな春の日を

鳴き尽くし、鳴きあかし、

また鳴き暮らさなければ気が済まんと見える。

その上どこまでも登って行く、

いつまでも登って行く。

雲雀はきっと雲の中で死ぬに相違ない。

登り詰めた揚句は、

流れて雲に入って、

漂うているうちに形は消えてなくなって、

ただ声だけが空の裡に残るのかもしれない。
"""
result(raw_txt)

       夏目漱石     芥川龍之介       森鴎外      宮沢賢治
0  0.116199  0.000063  0.000002  0.000007


'夏目漱石'

In [134]:
raw_txt = """
苦難が大きすぎて、
自分ひとりの力で支え切れない場合には、
家族から身を隠して一人で泣きなさい。

そして、苦悩を涙とともに洗い流したら、
頭をあげて胸を張り、
家族を激励するために家に戻りなさい。
"""
result(raw_txt)

       夏目漱石     芥川龍之介       森鴎外      宮沢賢治
0  0.000086  0.000445  0.000222  0.000005


'芥川龍之介'

In [135]:
raw_txt = """
それがしの宮の催したまいし星が岡茶寮のドイツ会に、洋行がえりの将校次をおうて身の上ばなしせしときのことなりしが、こよいはおん身が物語聞くべきはずなり、殿下も待ちかねておわすればとうながされて、まだ大尉になりてほどもあらじと見ゆる小林という少年士官、口にくわえし巻煙草まきたばこ取りて火鉢ひばちの中へ灰ふり落して語りははじめぬ。
"""
result(raw_txt)

       夏目漱石     芥川龍之介       森鴎外      宮沢賢治
0  0.000048  0.000001  0.000019  0.000005


'夏目漱石'

In [136]:
raw_txt = """
九月はじめの秋の空は、きょうしもここにまれなるあい色になりて、空気透すきとおりたれば、残るくまなくあざやかに見ゆるこの群れの真中まなかに、馬車一輛とどめさせて、年若き貴婦人いくたりか乗りたれば、さまざまの衣の色相映じて、花一叢そう、にしき一団、目もあやに、立ちたる人の腰帯シェルペ、坐りたる人の帽のひもなどを、風ひらひらと吹きなびかしたり。そのかたわらに馬立てたる白髪の翁おきなは角つのボタンどめにせし緑の猟人服かりうどふくに、うすき褐かちいろの帽をいただけるのみなれど、なにとなく由よしありげに見ゆ。すこし引き下がりて白き駒こま控えたる少女、わが目がねはしばしこれにとどまりぬ。鋼鉄はがねいろの馬のり衣ごろも裾長すそながに着て、白き薄絹巻きたる黒帽子をかぶりたる身の構えけだかく、いまかなたの森蔭より、むらむらと打ち出でたる猟兵の勇ましさ見んとて、人々騒げどかえりみぬさま心憎し。
「殊ことなるかたに心とどめたもうものかな」といいて軽くわが肩をうちし長き八字髭ひげの明色なる少年士官は、おなじ大隊の本部につけられたる中尉にて、男爵フォン、メエルハイムという人なり。「かしこなるはわが識れるデウベンの城のぬしビュロオ伯が一族なり。本部のこよいの宿はかの城と定まりたれば、君も人々に交わりたもうたつきあらん」といいおわるとき、猟兵ようようわが左翼に迫るを見て、メエルハイムは駈け去りぬ。この人とわが交わりそめしは、まだ久しからぬほどなれど、よき性さがとおもわれぬ。
"""
result(raw_txt)

       夏目漱石     芥川龍之介       森鴎外      宮沢賢治
0  0.000001  0.000318  0.007525  0.000015


'森鴎外'

In [137]:
raw_txt = """
ぼんやりとした不安
"""
result(raw_txt)

       夏目漱石     芥川龍之介       森鴎外      宮沢賢治
0  0.001286  0.001987  0.000394  0.000179


'芥川龍之介'

In [141]:
raw_txt = """
もうけつしてさびしくはない
なんべんさびしくないと云つたとこで
またさびしくなるのはきまつてゐる
"""
result(raw_txt)

           夏目漱石     芥川龍之介       森鴎外      宮沢賢治
0  9.799999e-07  0.004331  0.000437  0.202875


'宮沢賢治'

### 参考文献

#### Ngramについて
- http://www.denzow.me/entry/2017/10/29/121017

#### CNN、青空データの扱いについて
※このテーマで取り組む途中でかなり近いテーマの記事が見つかってしまったので、CNNではこれを参考にして実装している。
- https://qiita.com/bokeneko/items/c0f0ce60a998304400c8
- https://qiita.com/cvusk/items/c1342dd0fff16dc37ddf